 # Opis notatnika

 Ten notatnik inicjuje naszą pracę nad warsztatem końcowym. Naszym zadaniem tutaj jest pobranie udostępnionych nam danych do obszaru roboczego, które w następnym kroku wgramy na naszą bazę danych. Ich obróbka oraz analiza zostanie przeprowadzona w specjalnie do tego celu przygotowanych kolejnych notatnikach.

 Na potrzeby tego warsztatu został stworzony dedykowany serwis API, który dostępny jest pod adresem: https://api-datalab.coderslab.com/api/v2. Dodatkowo udostępniona została dokumentacja, z którą można zapoznać się tutaj: [klik](https://api-datalab.coderslab.com/v2/docs/).

 > Dokumentacja jest czysto techniczna i ma na celu prezentację dostępnych endpointów wraz ze zwracanym typem. W celu przetestowania należy kliknąć przysisk `Authorize`, podać token (dostępny poniżej), a następnie `Try it out!` oraz uzupełnić wymagane pola (parametry requesta).

 Zgodnie z dokumentacją stwierdzamy, że udostępnione zostały nam 4 endpointy:
 - `airport` - dane o lotnisku,
 - `weather` - informacje o zarejestrowaniej pogodzie na lotnisku danego dnia,
 - `aircraft` - dane o samolotach
 - `flights` - dane o wylotach z danego lotniska per dzień.

 Wszystkie te źródła musimy pobrać, aby być w stanie wykonać całość warsztatu. W celu pobrania informacji, gdzie wymagany jest paramatr `airportId`, posłużymy się listą z pliku `airports.csv`.

 Przy wykonywaniu tego zadania możesz posłużyć się tym tokenem: `iKRsQ8vdqgT903o2vH1rsejOeQ0F7YC9TvutH6Wk`.

 ### Uwagi
 - Ze względów ćwiczeniowych, konstrukcja poszczególnych endpointów jest różna – w trakcie pracy dokładnie przyjrzyj się, w jaki sposób należy wykonać zapytanie, aby otrzymać odpowiedź.
 - Pamiętaj o dodaniu `sleep` pomiędzy poszczególnymi wywołaniami endpoint.
 - Limit wywołań API to 1000/min, zadbaj o nieprzekroczenie tego limitu – w przeciwnym wypadku będzie zwracany błąd 429.

 # Konfiguracja notatnika

 Tutaj zaimportuj wymagane biblioteki

In [3]:
import requests
import time
import pandas as pd
import json
from datetime import datetime, timedelta
import glob

 Tutaj zdefiniuj paramatry połączenia do API

In [4]:
api_key = 'iKRsQ8vdqgT903o2vH1rsejOeQ0F7YC9TvutH6Wk'
api_url = 'https://api-datalab.coderslab.com/api/v2/'

auth = {'authorization': api_key}

 Tutaj wczytaj plik `airports.csv` i dostosuj do dalszych kroków w celu pobierania z kolejnych endpointów. Lista lotnisk jest dostępna w kolumnie `origin_airport_id`.

In [6]:
df_airports = pd.read_csv(r'..\data\airports.csv', encoding = 'UTF-8')
df_airports.shape

(364, 1)

 # Pobieranie `Airport`
 Zapoznaj się z dokumentacją endpointu `airport`, a następnie pobierz dane dot. poszczególnych lotnisk. Wyniki tego kroku zapisz do ramki `airport_df`, a następnie zapisz do pliku `csv`.

 ### Wskazówki
 - Nie wszystkie lotniska dostępne w pliku `airports.csv`, są dostępne w endpoint. Zadbaj o odpowiednie obsłużenie takiej sytuacji,
 - Do skonwertowania wyników przydatna może okazać się metoda `Pandas` - [from_records](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.from_records.html),
 - Artykuł LMS: `Python - analiza danych > Dzień 4 - API > Uwierzytelnianie`
 - Artykuł LMS: `Python - analiza danych > Przygotowanie do zjazdu 2`

 Tutaj pobierz dane z endpoint'u `airport`

In [7]:
## test if the API/code works:
# response = requests.get(f'{api_url}airport/11638', headers=auth)
# print(f"Status żądania: {response.status_code}")
# print(f"Odpowiedź: {response.text}")

all_airport_ids = df_airports['origin_airport_id']

data = []

for airport_id in all_airport_ids:
    response = requests.get(f'{api_url}airport/{airport_id}', headers=auth)
    if response.status_code == 200:
        airport_info = json.loads(response.text)
        airport_data = {
            'ORIGIN_AIRPORT_ID': airport_info.get('ORIGIN_AIRPORT_ID'),
            'DISPLAY_AIRPORT_NAME': airport_info.get('DISPLAY_AIRPORT_NAME'),
            'ORIGIN_CITY_NAME': airport_info.get('ORIGIN_CITY_NAME'),
            'NAME': airport_info.get('NAME')
        }
        data.append(airport_data)

airport_df = pd.DataFrame(data)
airport_df.head()

,ORIGIN_AIRPORT_ID,DISPLAY_AIRPORT_NAME,ORIGIN_CITY_NAME,NAME
0,11638,Fresno Air Terminal,"Fresno, CA","FRESNO YOSEMITE INTERNATIONAL, CA US"
1,13342,General Mitchell Field,"Milwaukee, WI","MILWAUKEE MITCHELL AIRPORT, WI US"
2,13244,Memphis International,"Memphis, TN","MEMPHIS INTERNATIONAL AIRPORT, TN US"
3,15096,Syracuse Hancock International,"Syracuse, NY","SYRACUSE HANCOCK INTERNATIONAL AIRPORT, NY US"
4,10397,Atlanta Municipal,"Atlanta, GA",ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...


In [8]:
airport_df.shape

(97, 4)

 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić czy ta część została poprawnie wykonana

In [9]:
airport_df_expected_shape = (97, 4)
assert airport_df_expected_shape == airport_df.shape

 Tutaj zapisz ramkę `airport_df` do pliku `airport_list.csv`

In [10]:
airport_df.to_csv(r'..\data\raw\airport_list.csv', index=False)

 # Pobieranie `Weather`
 Zapoznaj się z dokumentacją endpotu `Weather`, następnie pobierz dane dotyczące zarejestrowanej pogody na poszczególnych lotniskach. Wyniki zapisz do ramki `weather_df`, a później do pliku `airport_weather.csv`.

 Wskazówki:
 - Ze względu na wolumen danych, które tutaj się pobiorą, odradzamy zapisywanie danych bezpośrednio do ramki. Rekomendujemy podejście podobne do tego z warsztatu na kursie `Python - analiza danych` - `Dzień 10 - Warsztat > Warsztat > Scrapowanie danych`, czyli stworzenie listy, a następnie przekonwertowanie jej w postać ramki.
 - Data początkowa danych to `2019-01-01`, zaś data końcowa to `2020-03-31`, czyli 15 miesięcy,
 - Ze względu na czas, jaki ten krok będzie się wykonywał, warto dodać w pętli instrukcję (lub kilka) `print`, aby monitorować przebieg wykonywania tego kroku.
 - Przy dodawaniu miesięcy do daty może przydać się metoda [relativedelta](https://www.geeksforgeeks.org/python-get-month-from-year-and-weekday/).

In [11]:
start_date = datetime(2019, 1, 1)
end_date = datetime(2020, 3, 31)

current_date = start_date
unique_dates = set()

while current_date <= end_date:
    formatted_date = current_date.strftime('%Y-%m')
    unique_dates.add(formatted_date)
    current_date += timedelta(days=1)

unique_date_list = list(unique_dates)
unique_date_list.sort()  # Sort the unique dates if needed

data_weather = []

for date in unique_date_list:
    response = requests.get(f'{api_url}airportWeather?date={date}', headers=auth)
    if response.status_code == 200:
        x=response.json()
        data_weather += x


airport_weather_df = pd.DataFrame(data_weather)
airport_weather_df.head()

,WT18,STATION,NAME,DATE,AWND,PRCP,SNOW,SNWD,TAVG,TMAX,...,PGTM,WT10,WESD,SN32,SX32,PSUN,TSUN,TOBS,WT07,WT11
0,None,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,2019-01-01,4.70,0.14,0.0,0.0,64.0,66.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,None,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,2019-01-02,4.92,0.57,0.0,0.0,56.0,59.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,None,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,2019-01-03,5.37,0.15,0.0,0.0,52.0,55.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,None,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,2019-01-04,12.08,1.44,0.0,0.0,56.0,66.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,None,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,2019-01-05,13.42,0.00,0.0,0.0,49.0,59.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
airport_weather_df.shape

(46226, 33)

 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [13]:
airport_weather_df_expected_shape = (46226, 33)
assert airport_weather_df_expected_shape == airport_weather_df.shape

 ## Zapis do pliku
 Tutaj zapisz ramkę `weather_df` do pliku `airport_weather.csv` w katalogu `data/raw`

In [14]:
airport_weather_df.to_csv(r'..\data\raw\airport_weather.csv', index=False)

 # Pobranie `Aircraft`
 Zapoznaj się z dokumentacją endpointu `aircraft`, a następnie pobierz dane produkcyjne samolotów. Wyniki zapisz do ramki `aircraft_df`, a następnie zapisz do pliku `aircraft.csv`.


In [15]:
response = requests.get(f'{api_url}aircraft', headers=auth)

aircraft_df = pd.DataFrame(response.json())
aircraft_df.head()


,MANUFACTURE_YEAR,TAIL_NUM,NUMBER_OF_SEATS
0,1944,N54514,0.0
1,1945,N1651M,0.0
2,1953,N100CE,0.0
3,1953,N141FL,0.0
4,1953,N151FL,0.0


 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [16]:
aircraft_df_expected_shape = (7383, 3)
assert aircraft_df_expected_shape == aircraft_df.shape

 ## Zapis do pliku
 Tutaj zapisz ramkę `aircraft_df` do pliku `aircraft.csv` w katalogu `data/raw`

In [17]:
aircraft_df.to_csv(r'..\data\raw\aircraft.csv', index=False)

 # Pobranie `Flight`
 Zapoznaj się z dokumentacją endpointu `flights`, następnie pobierz dane dotyczące ruchu lotniczego. Wyniki zapisz do ramki `flight_df`, a później do pliku `flight.csv`.

 Wskazówki:
 - Zwróć szczególną uwagę na konstrukcję endpointa,
 - Ze względu na wolumen danych, które tutaj się pobiorą, odradzamy zapisywanie danych bezpośrednio do ramki. Rekomendujemy podejście podobne do tego, z warsztatu na kursie `Python - analiza danych` - `Dzień 10 - Warsztat > Warsztat > Scrapowanie danych`,
 - Data początkowa danych to `2019-01-01`, zaś końcowa to `2020-03-31`, czyli 456 dni,
 - Ze względu na czas, jaki ten krok będzie się wykonywał, warto dodać w pętli instrukcję (lub kilka) `print`, aby monitorować przebieg wykonywania tego kroku,
 - W przypadku, gdy nie ma dostępnych danych dla danego lotniska, API zwraca kod [204](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/204), w ten sposób możesz pominąć lotniska, dla których dane nie są dostępne,
 - Pobranie całości danych zajmuje dłuższą chwilę, zanim włączysz pętle dla wszystkich danych, sprawdź pobieranie danych dla jednego, dwóch lotnisk aby uniknąć frustracji.

In [43]:
start_date = datetime(2019, 1, 1)
end_date = datetime(2020, 3, 31)

current_date = start_date
unique_dates = set()

while current_date <= end_date:
    formatted_date = current_date.strftime('%Y-%m')
    unique_dates.add(formatted_date)
    current_date += timedelta(days=1)

unique_date_list = list(unique_dates)
unique_date_list.sort()  # Sort the unique dates

# getting the airport id list from airports.csv
with open(r'..\data\airports.csv') as file:
    airport_id_list = file.read().split('\n')[1:-1] # all airport ids without the column header(0) and last null value (-1)

for index, airportId in enumerate(airport_id_list):
    print(f"Processing airport ID: {airportId}, {index+1} out of {len(airport_id_list)}") # index+1 because python starts from 0
    for date in unique_date_list:
        response = requests.get(f'{api_url}flight?airportId={airportId}&date={date}', headers=auth)
        if response.status_code == 200:
            rows = response.json()
            if rows:
                # Check if data is not empty
                file_path = f'..\\data\\raw\\flight_outputs\\{airportId}_output.json'
                with open(file_path, 'a') as output_file:
                    for row in rows:
                        output_file.write(json.dumps(row)+'\n')
            else:
                print(f"No data received for airport {airportId} on date {date}")
        elif response.status_code == 204:
            pass
            #print(f"No data available for airport {airportId} on date {date}")
        else:
            print(f"Unexpected status code: {response.status_code}")
        time.sleep(0.06)


Processing airport ID: 11638, 0 out of 97
No data available for airport 11638 on date 2019-01
No data available for airport 11638 on date 2019-02
No data available for airport 11638 on date 2019-03
No data available for airport 11638 on date 2019-04
No data available for airport 11638 on date 2019-05
No data available for airport 11638 on date 2019-06
No data available for airport 11638 on date 2019-07
No data available for airport 11638 on date 2019-08
No data available for airport 11638 on date 2019-09
No data available for airport 11638 on date 2019-10
No data available for airport 11638 on date 2019-11
No data available for airport 11638 on date 2019-12
No data available for airport 11638 on date 2020-01
No data available for airport 11638 on date 2020-02
No data available for airport 11638 on date 2020-03
Processing airport ID: 13342, 1 out of 97
No data available for airport 13342 on date 2019-01
No data available for airport 13342 on date 2019-02
No data available for airport 13

In [5]:
# Get a list of file paths using glob
file_paths = glob.glob(r"..\data\raw\flight_outputs\*.json") 

total_files = len(file_paths)
# Loop through the list of file paths and read them into separate DataFrames
dfs = []
for index, file_path in enumerate(file_paths): #enumerate returns a tuple with index and file_path
    print(f"Processing file nr {index+1} out of {total_files}")
    dfs.append(pd.read_json(file_path, lines=True)) #lines=True reads each line in json file as a separate row in dataframe

flight_df = pd.concat(dfs, ignore_index=True)

flight_df.shape
flight_df.head()

Processing file nr 1 out of 39
Processing file nr 2 out of 39
Processing file nr 3 out of 39
Processing file nr 4 out of 39
Processing file nr 5 out of 39
Processing file nr 6 out of 39
Processing file nr 7 out of 39
Processing file nr 8 out of 39
Processing file nr 9 out of 39
Processing file nr 10 out of 39
Processing file nr 11 out of 39
Processing file nr 12 out of 39
Processing file nr 13 out of 39
Processing file nr 14 out of 39
Processing file nr 15 out of 39
Processing file nr 16 out of 39
Processing file nr 17 out of 39
Processing file nr 18 out of 39
Processing file nr 19 out of 39
Processing file nr 20 out of 39
Processing file nr 21 out of 39
Processing file nr 22 out of 39
Processing file nr 23 out of 39
Processing file nr 24 out of 39
Processing file nr 25 out of 39
Processing file nr 26 out of 39
Processing file nr 27 out of 39
Processing file nr 28 out of 39
Processing file nr 29 out of 39
Processing file nr 30 out of 39
Processing file nr 31 out of 39
Processing file n

 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [6]:
flight_df_expected_shape = (1386120, 27)
assert flight_df_expected_shape == flight_df.shape

 ## Zapis do pliku
 Tutaj zapisz ramkę `flight_df` do pliku `flight.csv` w katalogu `data/raw`

In [7]:
flight_df.to_csv(r'..\data\raw\flight.csv', index=False)

 # Podsumowanie
 W tym notatniku wykonaliśmy podstawowy krok w analizie danych - pozyskaliśmy je. Są gotowe do dalszej pracy, czyli możemy załadować je na bazę danych, a następnie zapoznać się z tym, jakie informacje ze sobą niosą. Kolejne notatniki będą służyły właśnie tym celom.

In [8]:
msg = "Wszystko wygląda OK :) Możesz przejść do kolejnego kroku."
print(msg)

Wszystko wygląda OK :) Możesz przejść do kolejnego kroku.
